# Building text classifier with Differential Privacy

In this tutorial we will train a text classifier with Differential Privacy by taking a model pre-trained on public text data and fine-tuning it for a different task.

When training a model with differential privacy, we almost always face a trade-off between model size and accuracy on the task. The exact details depend on the problem, but a rule of thumb is that the fewer parameters the model has, the easier it is to get a good performance with DP.

Most state-of-the-art NLP models are quite deep and large (e.g. [BERT-base](https://github.com/google-research/bert) has over 100M parameters), which makes task of training text model on a private datasets rather challenging.

One way of addressing this problem is to divide the training process into two stages. First, we will pre-train the model on a public dataset, exposing the model to generic text data. Assuming that the generic text data is public, we will not be using differential privacy at this step. Then, we freeze most of the layers, leaving only a few upper layers to be trained on the private dataset using DP-SGD. This way we can get the best of both worlds - we have a deep and powerful text understanding model, while only training a small number of parameters with differentially private algorithm.

In this tutorial we will take the pre-trained [BERT-base](https://github.com/google-research/bert) model and fine-tune it to recognize textual entailment on the [SNLI](https://nlp.stanford.edu/projects/snli/) dataset.

## Dataset

In [1]:
from google.colab import files
CS_file = files.upload()

Saving CovidSentimentData.csv to CovidSentimentData.csv


In [2]:
import io
import pandas as pd

data = pd.read_csv(io.BytesIO(CS_file['CovidSentimentData.csv']))

In [3]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(data, random_state=42, test_size=0.2)

## Model

BERT (Bidirectional Encoder Representations from Transformers) is state of the art approach to various NLP tasks. It uses a Transformer architecture and relies heavily on the concept of pre-training. 

We'll use a pre-trained BERT-base model, provided in huggingface [transformers](https://github.com/huggingface/transformers) repo.
It gives us a pytorch implementation for the classic BERT architecture, as well as a tokenizer and weights pre-trained on a public English corpus (Wikipedia).

Please follow these [installation instrucitons](https://github.com/huggingface/transformers#installation) before proceeding.

In [4]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 5.0 MB/s 
     |████████████████████████████████| 596 kB 70.8 MB/s 
     |████████████████████████████████| 61 kB 627 kB/s 
     |████████████████████████████████| 3.3 MB 81.7 MB/s 
     |████████████████████████████████| 895 kB 61.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [5]:
from transformers import BertConfig, BertTokenizer, BertForSequenceClassification

model_name = "bert-base-cased"

config = BertConfig.from_pretrained(
    model_name,
    num_labels=2,
)
tokenizer = BertTokenizer.from_pretrained(
    "bert-base-cased",
    do_lower_case=True,
)
model = BertForSequenceClassification.from_pretrained(
    "bert-base-cased",
    config=config,
)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

The model has the following structure. It uses a combination of word, positional and token *embeddings* to create a sequence representation, then passes the data through 12 *transformer encoders* and finally uses a *linear classifier* to produce the final label.
As the model is already pre-trained and we only plan to fine-tune few upper layers, we want to freeze all layers, except for the last encoder and above (`BertPooler` and `Classifier`).

In [6]:
trainable_layers = [model.bert.encoder.layer[-1], model.bert.pooler, model.classifier]
total_params = 0
trainable_params = 0

for p in model.parameters():
        p.requires_grad = False
        total_params += p.numel()

for layer in trainable_layers:
    for p in layer.parameters():
        p.requires_grad = True
        trainable_params += p.numel()

print(f"Total parameters count: {total_params}") # ~108M
print(f"Trainable parameters count: {trainable_params}") # ~7M

Total parameters count: 108311810
Trainable parameters count: 7680002


Thus, by using pre-trained model we reduce the number of trainable params from over 100 millions to just above 7.5 millions. This will help both performance and convergence with added noise.

## Prepare the data

Before we begin training, we need to preprocess the data and convert it to the format our model expects. 

(Note: it'll take 5-10 minutes to run on a laptop)

In [7]:
LABEL_LIST = [0,1]
MAX_SEQ_LENGHT = 128

import torch
import transformers
from torch.utils.data import TensorDataset
from transformers.data.processors.utils import InputExample
from transformers.data.processors.glue import glue_convert_examples_to_features


def _create_examples(df, set_type):
    """ Convert raw dataframe to a list of InputExample. Filter malformed examples
    """
    examples = []
    for index, row in df.iterrows():
        if row['target'] not in LABEL_LIST:
            continue
        if not isinstance(row['Tweet'], str):
            continue
            
        guid = f"{index}-{set_type}"
        examples.append(
            InputExample(guid=guid, text_a=row['Tweet'], label=row['target']))
    return examples

def _df_to_features(df, set_type):
    """ Pre-process text. This method will:
    1) tokenize inputs
    2) cut or pad each sequence to MAX_SEQ_LENGHT
    3) convert tokens into ids
    
    The output will contain:
    `input_ids` - padded token ids sequence
    `attention mask` - mask indicating padded tokens
    `token_type_ids` - mask indicating the split between premise and hypothesis
    `label` - label
    """
    examples = _create_examples(df, set_type)
    
    #backward compatibility with older transformers versions
    legacy_kwards = {}
    from packaging import version
    if version.parse(transformers.__version__) < version.parse("2.9.0"):
        legacy_kwards = {
            "pad_on_left": False,
            "pad_token": tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0],
            "pad_token_segment_id": 0,
        }
    
    return glue_convert_examples_to_features(
        examples=examples,
        tokenizer=tokenizer,
        label_list=LABEL_LIST,
        max_length=MAX_SEQ_LENGHT,
        output_mode="classification",
        **legacy_kwards,
    )

def _features_to_dataset(features):
    """ Convert features from `_df_to_features` into a single dataset
    """
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_attention_mask = torch.tensor(
        [f.attention_mask for f in features], dtype=torch.long
    )
    all_token_type_ids = torch.tensor(
        [f.token_type_ids for f in features], dtype=torch.long
    )
    all_labels = torch.tensor([f.label for f in features], dtype=torch.long)
    dataset = TensorDataset(
        all_input_ids, all_attention_mask, all_token_type_ids, all_labels
    )

    return dataset

train_features = _df_to_features(df_train, "train")
test_features = _df_to_features(df_test, "test")

train_dataset = _features_to_dataset(train_features)
test_dataset = _features_to_dataset(test_features)

/usr/local/lib/python3.7/dist-packages/transformers/data/processors/glue.py:67: FutureWarning: This function will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING.format("function"), FutureWarning)


## Choosing batch size

Let's talk about batch sizes for a bit.

In addition to all the considerations you normally take into account when choosing batch size, training model with DP adds another one - privacy cost. 

Because of the threat model we assume and the way we add noise to the gradients, larger batch sizes (to a certain extent) generally help convergence. We add the same amount of noise to each gradient update (scaled to the norm of one sample in the batch) regardless of the batch size. What this means is that as the batch size increases, the relative amount of noise added decreases. while preserving the same epsilon guarantee. 

You should, however, keep in mind that increasing batch size has its price in terms of epsilon, which grows at `O(sqrt(batch_size))` as we train (therefore larger batches make it grow faster). The good strategy here is to experiment with multiple combinations of `batch_size` and `noise_multiplier` to find the one that provides best possible quality at acceptable privacy guarantee.

There's another side to this - memory. Opacus computes and stores *per sample* gradients, so for every normal gradient, Opacus will store `n=batch_size` per-sample gradients on each step, thus increasing the memory footprint by at least `O(batch_size)`. In reality, however, the peak memory requirement is `O(batch_size^2)` compared to non-private model. This is because some intermediate steps in per sample gradient computation involve operations on two matrices, each with batch_size as one of the dimensions.

The good news is, we can pick the most appropriate batch size, regardless of memory constrains. Opacus has built-in support for *virtual* batches. Using it we can separate physical steps (gradient computation) and logical steps (noise addition and parameter updates): use larger batches for training, while keeping memory footprint low. Below we will specify two constants:

- `BATCH_SIZE` defines the maximum batch size we can afford from a memory standpoint, and only affects computation speed
- `VIRTUAL_BATCH_SIZE`, on the other hand, is equivalent to normal batch_size in the non-private setting, and will affect convergence and privacy guarantee.



In [8]:
BATCH_SIZE = 4
VIRTUAL_BATCH_SIZE = 32
assert VIRTUAL_BATCH_SIZE % BATCH_SIZE == 0 # VIRTUAL_BATCH_SIZE should be divisible by BATCH_SIZE
N_ACCUMULATION_STEPS = int(VIRTUAL_BATCH_SIZE / BATCH_SIZE)

In [9]:
!pip install opacus==0.15.0

     |████████████████████████████████| 125 kB 5.1 MB/s 


In [10]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from opacus.utils.uniform_sampler import UniformWithReplacementSampler

SAMPLE_RATE = BATCH_SIZE / len(train_dataset)

train_sampler=UniformWithReplacementSampler(
    num_samples=len(train_dataset),
    sample_rate=SAMPLE_RATE,
)
train_dataloader = DataLoader(train_dataset, batch_sampler=train_sampler)

test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=BATCH_SIZE)

## Training

In [11]:
import torch

# Move the model to appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Set the model to train mode (HuggingFace models load in eval mode)
model = model.train()

# Define optimizer

optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, eps=1e-8)
#lambda1 = lambda epoch: 0.65 ** epoch
#scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer=optimizer, lr_lambda=lambda1)

First we specify some training parameters ready to run the training loop for three epochs

In [12]:
EPOCHS = 3
LOGGING_INTERVAL = 100 # once every how many steps we run evaluation cycle and report metrics
EPSILON = 5.5
DELTA = 1 / len(train_dataloader) # Parameter for privacy accounting. Probability of not achieving privacy guarantees

Let’s now define the evaluation cycle.

In [13]:
import numpy as np
from sklearn.metrics import accuracy_score
from tqdm.notebook import tqdm

def accuracy(preds, labels):
    return (preds == labels).mean()

# define evaluation cycle
def evaluate(model):    
    model.eval()

    loss_arr = []
    pred_arr = []
    label_arr = []
    accuracy_arr = []
    
    for batch in test_dataloader:
        batch = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                      'token_type_ids': batch[2],
                      'labels':         batch[3]}

            outputs = model(**inputs)
            loss, logits = outputs[:2]
            loss = outputs[0]
            logits = torch.abs(outputs[1])
            preds = np.argmax(logits.detach().cpu().numpy(), axis=1)
            labels = inputs['labels'].detach().cpu().numpy()
            
            loss_arr.append(loss.item())
            accuracy_arr.append(accuracy(preds, labels))

    model.train()
    
    return np.mean(loss_arr), np.mean(accuracy_arr)

Next we will define and attach PrivacyEngine. There are two parameters you need to consider here:

- `noise_multiplier`. It defines the trade-off between privacy and accuracy. Adding more noise will provide stronger privacy guarantees, but will also hurt model quality.  In this run, the PrivacyEngine will determine this value based on the target values of `EPSILON`, `DELTA` and `EPOCHS`.  For the default settings, this will set `noise_multiplier` to about 0.4. 
- `max_grad_norm`. Defines the maximum magnitude of L2 norms to which we clip per sample gradients. There is a bit of tug of war with this threshold: on the one hand, a low threshold means that we will clip many gradients, hurting convergence, so we might be tempted to raise it. However, recall that we add noise with `std=noise_multiplier * max_grad_norm` so we will pay for the increased threshold with more noise. In most cases you can rely on the model being quite resilient to clipping (after the first few iterations your model will tend to adjust so that its gradients stay below the clipping threshold), so you can often just keep the default value (`=1.0`) and focus on tuning `batch_size` and `noise_multiplier` instead. That being said, sometimes clipping hurts the model so it may be worth experimenting with different clipping thresholds, like we are doing in this tutorial.

These two parameters define the scale of the noise we add to gradients: the noise will be sampled from a Gaussian distribution with `std=noise_multiplier * max_grad_norm`.


In [14]:
from opacus import PrivacyEngine

MAX_GRAD_NORM = 0.1

privacy_engine = PrivacyEngine(
    module=model,
    sample_rate=SAMPLE_RATE * N_ACCUMULATION_STEPS,
    target_delta = DELTA,
    target_epsilon = EPSILON, 
    epochs = EPOCHS,
    max_grad_norm=MAX_GRAD_NORM,
)
privacy_engine.attach(optimizer)

/usr/local/lib/python3.7/dist-packages/opacus/privacy_engine.py:760: UserWarning: A ``sample_rate`` has been provided.Thus, the provided ``batch_size``and ``sample_size`` will be ignored.
  "A ``sample_rate`` has been provided."
/usr/local/lib/python3.7/dist-packages/opacus/privacy_engine.py:237: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_rng`` turned on.
  "Secure RNG turned off. This is perfectly fine for experimentation as it allows "


Now we can train the model.

In [15]:
for epoch in range(1, EPOCHS+1):

    torch.manual_seed(19)
    torch.cuda.manual_seed_all(19)

    losses = []
    model.train()

    for step, batch in enumerate(tqdm(train_dataloader)):
        
        batch = tuple(t.to(device) for t in batch)
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'token_type_ids': batch[2],
                  'labels':         batch[3]}

        outputs = model(**inputs) # output = loss, logits, hidden_states, attentions

        loss = outputs[0]
        loss.backward()
        
        losses.append(loss.item())

        # We process small batches of size BATCH_SIZE, 
        # until they're accumulated to a batch of size VIRTUAL_BATCH_SIZE.
        # Only then we make a real `.step()` and update model weights
        if (step + 1) % N_ACCUMULATION_STEPS == 0 or step == len(train_dataloader) - 1:
            optimizer.step()
        else:
            optimizer.virtual_step()

        if step > 0 and step % LOGGING_INTERVAL == 0:
            train_loss = np.mean(losses)
            eps, alpha = optimizer.privacy_engine.get_privacy_spent(DELTA)

            eval_loss, eval_accuracy = evaluate(model)

            print(
                f"Epoch: {epoch} | "
                f"Step: {step} | "
                f"Train loss: {train_loss:.3f} | "
                f"Eval loss: {eval_loss:.3f} | "
                f"Eval accuracy: {eval_accuracy:.3f} | "
                f"ɛ: {eps:.2f} (α: {alpha})"
            )

  0%|          | 0/3824 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:1025: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


Epoch: 1 | Step: 100 | Train loss: 0.885 | Eval loss: 1.565 | Eval accuracy: 0.828 | ɛ: 2.95 (α: 3.3)
Epoch: 1 | Step: 200 | Train loss: 1.289 | Eval loss: 1.728 | Eval accuracy: 0.828 | ɛ: 3.14 (α: 3.2)
Epoch: 1 | Step: 300 | Train loss: 1.508 | Eval loss: 1.346 | Eval accuracy: 0.828 | ɛ: 3.26 (α: 3.1)
Epoch: 1 | Step: 400 | Train loss: 1.437 | Eval loss: 1.199 | Eval accuracy: 0.828 | ɛ: 3.35 (α: 3.1)
Epoch: 1 | Step: 500 | Train loss: 1.421 | Eval loss: 1.323 | Eval accuracy: 0.828 | ɛ: 3.44 (α: 3.1)
Epoch: 1 | Step: 600 | Train loss: 1.417 | Eval loss: 1.272 | Eval accuracy: 0.828 | ɛ: 3.51 (α: 3.0)
Epoch: 1 | Step: 700 | Train loss: 1.410 | Eval loss: 1.227 | Eval accuracy: 0.828 | ɛ: 3.56 (α: 3.0)
Epoch: 1 | Step: 800 | Train loss: 1.392 | Eval loss: 1.285 | Eval accuracy: 0.828 | ɛ: 3.62 (α: 3.0)
Epoch: 1 | Step: 900 | Train loss: 1.382 | Eval loss: 1.322 | Eval accuracy: 0.828 | ɛ: 3.67 (α: 3.0)
Epoch: 1 | Step: 1000 | Train loss: 1.376 | Eval loss: 1.260 | Eval accuracy: 0.82

  0%|          | 0/3824 [00:00<?, ?it/s]

Epoch: 2 | Step: 100 | Train loss: 1.189 | Eval loss: 1.298 | Eval accuracy: 0.709 | ɛ: 4.55 (α: 2.7)
Epoch: 2 | Step: 200 | Train loss: 1.241 | Eval loss: 1.316 | Eval accuracy: 0.641 | ɛ: 4.57 (α: 2.7)
Epoch: 2 | Step: 300 | Train loss: 1.364 | Eval loss: 1.236 | Eval accuracy: 0.660 | ɛ: 4.59 (α: 2.7)
Epoch: 2 | Step: 400 | Train loss: 1.338 | Eval loss: 1.270 | Eval accuracy: 0.719 | ɛ: 4.61 (α: 2.7)
Epoch: 2 | Step: 500 | Train loss: 1.346 | Eval loss: 1.311 | Eval accuracy: 0.661 | ɛ: 4.63 (α: 2.7)
Epoch: 2 | Step: 600 | Train loss: 1.350 | Eval loss: 1.252 | Eval accuracy: 0.690 | ɛ: 4.65 (α: 2.7)
Epoch: 2 | Step: 700 | Train loss: 1.359 | Eval loss: 1.264 | Eval accuracy: 0.714 | ɛ: 4.67 (α: 2.7)
Epoch: 2 | Step: 800 | Train loss: 1.349 | Eval loss: 1.308 | Eval accuracy: 0.648 | ɛ: 4.69 (α: 2.7)
Epoch: 2 | Step: 900 | Train loss: 1.346 | Eval loss: 1.331 | Eval accuracy: 0.557 | ɛ: 4.71 (α: 2.7)
Epoch: 2 | Step: 1000 | Train loss: 1.348 | Eval loss: 1.280 | Eval accuracy: 0.35

  0%|          | 0/3824 [00:00<?, ?it/s]

Epoch: 3 | Step: 100 | Train loss: 1.223 | Eval loss: 1.401 | Eval accuracy: 0.168 | ɛ: 5.20 (α: 2.6)
Epoch: 3 | Step: 200 | Train loss: 1.293 | Eval loss: 1.383 | Eval accuracy: 0.171 | ɛ: 5.22 (α: 2.6)
Epoch: 3 | Step: 300 | Train loss: 1.404 | Eval loss: 1.287 | Eval accuracy: 0.170 | ɛ: 5.23 (α: 2.6)
Epoch: 3 | Step: 400 | Train loss: 1.368 | Eval loss: 1.322 | Eval accuracy: 0.168 | ɛ: 5.25 (α: 2.6)
Epoch: 3 | Step: 500 | Train loss: 1.379 | Eval loss: 1.381 | Eval accuracy: 0.165 | ɛ: 5.26 (α: 2.6)
Epoch: 3 | Step: 600 | Train loss: 1.384 | Eval loss: 1.331 | Eval accuracy: 0.162 | ɛ: 5.28 (α: 2.6)
Epoch: 3 | Step: 700 | Train loss: 1.399 | Eval loss: 1.304 | Eval accuracy: 0.160 | ɛ: 5.29 (α: 2.6)
Epoch: 3 | Step: 800 | Train loss: 1.384 | Eval loss: 1.352 | Eval accuracy: 0.158 | ɛ: 5.31 (α: 2.6)
Epoch: 3 | Step: 900 | Train loss: 1.381 | Eval loss: 1.381 | Eval accuracy: 0.158 | ɛ: 5.32 (α: 2.6)
Epoch: 3 | Step: 1000 | Train loss: 1.384 | Eval loss: 1.312 | Eval accuracy: 0.16

In [16]:
from google.colab import files
PS_file = files.upload()

Saving prediction_score.csv to prediction_score.csv


In [17]:
ps_data = pd.read_csv('/content/prediction_score.csv')

In [18]:
ps_data

,Tweet,target
0,Corona time gains We have no control over the ...,0
1,This is how Chinese people caught Corona Virus...,0
2,an empty Escalator at M Purple Line station. W...,0
3,I am working away from home to earn a living a...,0
4,The virus is too dangerous and may last for ma...,0


In [19]:
ps_features = _df_to_features(ps_data, "test")
ps_dataset = _features_to_dataset(ps_features)
ps_sampler = SequentialSampler(ps_dataset)
ps_dataloader = DataLoader(ps_dataset, sampler=ps_sampler, batch_size=5)

/usr/local/lib/python3.7/dist-packages/transformers/data/processors/glue.py:67: FutureWarning: This function will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING.format("function"), FutureWarning)


In [20]:
for batch in ps_dataloader:
        batch = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                      'token_type_ids': batch[2],
                      'labels':         batch[3]}

In [21]:
outputs = model(**inputs)
loss, logits = outputs[:2]
loss = outputs[0]
logits = torch.abs(outputs[1])
preds = np.argmax(logits.detach().cpu().numpy(), axis=1)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:1025: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


In [22]:
logits

tensor([[5.3435, 3.7709],
        [5.6878, 2.3265],
        [5.1261, 3.2730],
        [4.4799, 3.5603],
        [4.5701, 2.1722]], device='cuda:0', grad_fn=<AbsBackward0>)

In [23]:
preds

array([0, 0, 0, 0, 0])

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Nov 10 09:03:01 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    40W / 250W |  13595MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

For the test accuracy, after training for three epochs you should expect something close to the results below.

You can see that we can achieve quite strong privacy guarantee at epsilon=7.5 with a moderate accuracy cost of 11 percentage points compared to non-private model trained in a similar setting (upper layers only) and 16 points compared to best results we were able to achieve using the same architecture.

*NB: When not specified, DP-SGD is trained with upper layers only*

| Model | Noise multiplier | Batch size | Accuracy | Epsilon |
| --- | --- | --- | --- | --- |
| no DP, train full model | N/A | 32 | 90.1% | N/A |
| no DP, train upper layers only | N/A | 32 | 85.4% | N/A |
| DP-SGD | 1.0 | 32 | 70.5% | 0.7 |
| **DP-SGD (this tutorial)** | **0.4** | **32** | **74.3%** | **7.5** |
| DP-SGD | 0.3 | 32 | 75.8% | 20.7 |
| DP-SGD | 0.1 | 32 | 78.3% | 2865 |
| DP-SGD | 0.4 | 8 | 67.3% | 5.9 |